In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("power_training_data_1_morning_832_842_lathrop.csv")

In [3]:
data

,SN,timestamp,total_power,p0,p1,p2,p3,p4,p5,p6
0,0,2022-09-29T15:32:00.255000Z,0.903975,-0.874080,0.457777,2.000000,2.000000,2.000000,2.000000,0.775986
1,1,2022-09-29T15:32:00.383000Z,-0.850796,-0.842891,-0.557550,-0.575725,-0.536052,-0.493993,-0.426373,-0.502608
2,2,2022-09-29T15:32:00.511000Z,-0.868921,-0.855945,-0.601083,-0.579232,-0.535855,-0.499467,-0.435050,-0.513740
3,3,2022-09-29T15:32:00.639000Z,-0.881959,-0.875006,-0.602147,-0.578875,-0.561164,-0.473057,-0.437372,-0.524266
4,4,2022-09-29T15:32:00.767000Z,-0.881974,-0.873990,-0.602932,-0.574325,-0.540357,-0.502455,-0.434504,-0.523742
...,...,...,...,...,...,...,...,...,...,...
6028,6028,2022-09-29T15:44:59.483000Z,-0.885068,-0.875243,-0.603128,-0.580944,-0.565439,-0.503714,-0.436152,-0.524747
6029,6029,2022-09-29T15:44:59.611000Z,-0.884633,-0.875255,-0.603125,-0.581119,-0.563538,-0.500041,-0.437138,-0.524150
6030,6030,2022-09-29T15:44:59.739000Z,-0.882703,-0.875246,-0.601668,-0.581475,-0.563626,-0.500403,-0.435309,-0.519699
6031,6031,2022-09-29T15:44:59.867000Z,-0.883680,-0.875247,-0.603105,-0.579910,-0.565530,-0.500806,-0.436144,-0.521572


In [4]:
from influxdb import InfluxDBClient

In [32]:
client = InfluxDBClient(host="127.0.0.1", port=8086, database="lathrop")

x = client.query(
    "select * from raw where time>='2023-01-16T17:34:19.000Z' and time<'2023-01-16T17:34:48.000Z'"
)
data=pd.DataFrame(x.get_points())
print(len(data))

0


In [5]:
def query_betn(s_datetime,e_datetime,table="raw",client = InfluxDBClient(host="127.0.0.1", port=8086, database="lathrop")):
    x = client.query(
        f"select * from {table} where time>='{s_datetime}' and time<'{e_datetime}'"
    )
    data=pd.DataFrame(x.get_points())
    print(len(data))
    data.time=pd.to_datetime(data.time)
    return data

In [14]:
import yaml 
from pathlib import Path
import numpy as np
def read_params(folder_location):
    with open(folder_location/"params.yml","r") as fid:
        d=yaml.safe_load(fid)
    print(d["fundFreq"])
    return d

In [15]:
nor_data= [ 1.13106413, 0.02172424, -0.22792327, -0.24767245,  0.2529968,  -0.01294536, -0.36451498]
params=read_params(Path("/home/spoudel/All_Thesis/Thesis_program/Data/lathrop/location_1/"))

62.5


In [28]:
fundFreq=params["fundFreq"]
std=params["std"]
mean=params["mean"]
f = (
    np.array(
        [
            fundFreq,
            2 * fundFreq,
            3 * fundFreq,
            4 * fundFreq,
            5 * fundFreq,
            6 * fundFreq,
            7 * fundFreq,
        ]
    )
    ** 2
)

# np.array([(a*b+c)*d for a,b,c,d in zip(x,std,mean,f)])/1000000 
np.sum([(a*b+c)*d for a,b,c,d in zip(nor_data,std,mean,f)])/1000000

0.0003259537994661864

In [30]:
powers=np.array([0.0003259537995217399, 0.00037097925299087886, 0.0002694689375181375, 0.00031729496740775696, 0.0003282480123433774])

In [31]:
powers.argsort()

array([2, 3, 0, 4, 1])

In [36]:
clusters=np.array([[ 0.14519037, -0.18461004 ,-0.27475222 , 0.02691569,  1.47445884, -0.37297144,  0.00618826],
[ 0.07254911 ,-0.20665212 , 0.12000857  ,0.1527345 ,  0.06405563 , 1.53748828,  0.04532875],
[-0.06340674, -0.38889314 ,-0.29170255 ,-0.05354097, -0.33344382, -0.35590765, -0.07685777],
[-0.02739634 ,-0.30726716,  1.36301424 ,-0.13563769 ,-0.25065642, -0.21088818, -0.01745732],
[ 0.07341146 , 1.42665629, -0.29690881, -0.17843133, -0.18643052, -0.2787791, -0.1067004 ]])

In [42]:
idx=clusters.T[0].argsort()
clusters[idx]

array([[-0.06340674, -0.38889314, -0.29170255, -0.05354097, -0.33344382,
        -0.35590765, -0.07685777],
       [-0.02739634, -0.30726716,  1.36301424, -0.13563769, -0.25065642,
        -0.21088818, -0.01745732],
       [ 0.07254911, -0.20665212,  0.12000857,  0.1527345 ,  0.06405563,
         1.53748828,  0.04532875],
       [ 0.07341146,  1.42665629, -0.29690881, -0.17843133, -0.18643052,
        -0.2787791 , -0.1067004 ],
       [ 0.14519037, -0.18461004, -0.27475222,  0.02691569,  1.47445884,
        -0.37297144,  0.00618826]])